# Entrega #2

Con el objetivo de usar de la mejor forma posible la base de datos en la creación del modelo predictivo, se optó por manipular el archivo "train.csv" para hacer un preprocesamiento de la información.

Para ello, se siguieron las siguientes estrategias:

1. Se eliminaron dos columnas que no aportaban para el modelo:
  *   "FAMI_TIENEINTERNET.1": Está repetida y contiene la misma información que "FAMI_TIENEINTERNET"
  *   "ESTU_PRIVADO_LIBERTAD": Todos los datos que contiene esta columna arrojan un "No", por lo que es una variable que no influye en el comportamiento del modelo.



2.   Las columnas que contienen un Si/No, se cambiaron por valores booleanos, convirtiendose en True/False respectivamente. Esto para mejorar la eficiencia en la busqueda y manipulación de la información.

3. Se hizo uso de One Hot Encoding para columnas que contenían rangos para facilitar mejor el procesamiento de esa información que se ve combinada entre strings e integers.

4. Se hizo uso de la codificación ordinal para columnas que contenían rangos que seguían cierta jerarquia (en el caso del rendimiento)

5. Para columnas con datos faltantes, se imputaron los datos restantes haciendo uso de la moda (al ser columnas categoricas)





## Cargar Datos

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import os

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [ ]:
!kaggle competitions download -c udea-ai4eng-20241


100% 21.2M/21.2M [00:00<00:00, 120MB/s] 
100% 21.2M/21.2M [00:00<00:00, 109MB/s]


In [ ]:
!unzip udea-ai4eng-20241.zip

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
print(df.isnull().sum())

ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PRIVADO_LIBERTAD                 0
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_TIENEINTERNET.1              26629
FAMI_EDUCACIONMADRE               23664
RENDIMIENTO_GLOBAL                    0
dtype: int64


In [ ]:
print(df.columns)

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL'],
      dtype='object')


# Pre-Procesamiento de la Información:

## 1. Eliminar columnas innecesarias:

### FAMI_TIENEINTERNET (REPETIDA)

In [ ]:
columnas_redundantes = ['FAMI_TIENEINTERNET.1']

df = df.drop(columnas_redundantes, axis=1)


### ESTU_PRIVADO_LIBERTAD (Innecesaria)

In [ ]:
columna_privado_libertad = 'ESTU_PRIVADO_LIBERTAD'

df = df.drop(columna_privado_libertad, axis=1)

## 2. Pasar columnas de "Si/No" a "True/False":

In [ ]:
import pandas as pd

columnas_booleanas = ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
                      'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']

for columna in columnas_booleanas:
    print(f"Valores únicos en la columna '{columna}': {df[columna].unique()}")

for columna in columnas_booleanas:
    df[columna] = df[columna].map({'Si': True, 'No': False})

Valores únicos en la columna 'FAMI_TIENEINTERNET': ['Si' 'No' nan]
Valores únicos en la columna 'FAMI_TIENELAVADORA': ['Si' 'No' nan]
Valores únicos en la columna 'FAMI_TIENEAUTOMOVIL': ['Si' 'No' nan]
Valores únicos en la columna 'ESTU_PAGOMATRICULAPROPIO': ['No' 'Si' nan]
Valores únicos en la columna 'FAMI_TIENECOMPUTADOR': ['Si' 'No' nan]


## 3. Borrar Strings de Columnas:

In [ ]:
print(df['FAMI_ESTRATOVIVIENDA'].head(10))


0    Estrato 3
1    Estrato 3
2    Estrato 3
3    Estrato 4
4    Estrato 3
5    Estrato 5
6    Estrato 2
7    Estrato 2
8    Estrato 1
9    Estrato 5
Name: FAMI_ESTRATOVIVIENDA, dtype: object


In [ ]:
df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato ', '')

print(df['FAMI_ESTRATOVIVIENDA'].head(10))


0    3
1    3
2    3
3    4
4    3
5    5
6    2
7    2
8    1
9    5
Name: FAMI_ESTRATOVIVIENDA, dtype: object


## 4. Manipular Rangos

### Rangos de VALORMATRICULAUNIVERSIDAD

In [ ]:
print(df['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts())

ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      204048
Entre 2.5 millones y menos de 4 millones    127430
Menos de 500 mil                             80263
Entre 500 mil y menos de 1 millón            78704
Entre 4 millones y menos de 5.5 millones     69736
Más de 7 millones                            68014
Entre 5.5 millones y menos de 7 millones     38490
No pagó matrícula                            19528
Name: count, dtype: int64


In [ ]:
import pandas as pd


encoded_df = pd.get_dummies(df['ESTU_VALORMATRICULAUNIVERSIDAD'])

df = pd.concat([df, encoded_df], axis=1)

df.drop('ESTU_VALORMATRICULAUNIVERSIDAD', axis=1, inplace=True)

print(df)


            ID  PERIODO                  ESTU_PRGM_ACADEMICO  \
0       904256    20212                           ENFERMERIA   
1       645256    20212                              DERECHO   
2       308367    20203                MERCADEO Y PUBLICIDAD   
3       470353    20195           ADMINISTRACION DE EMPRESAS   
4       989032    20212                           PSICOLOGIA   
...        ...      ...                                  ...   
692495   25096    20195                             BIOLOGIA   
692496  754213    20212                           PSICOLOGIA   
692497  504185    20183  ADMINISTRACIÓN EN SALUD OCUPACIONAL   
692498  986620    20195                           PSICOLOGIA   
692499  933374    20195                           PSICOLOGIA   

       ESTU_PRGM_DEPARTAMENTO ESTU_HORASSEMANATRABAJA FAMI_ESTRATOVIVIENDA  \
0                      BOGOTÁ       Menos de 10 horas                    3   
1                   ATLANTICO                       0                    3 

### Rangos de ESTU_HORASSEMANA

In [ ]:
print(df['ESTU_HORASSEMANATRABAJA'].value_counts())

ESTU_HORASSEMANATRABAJA
Más de 30 horas        249352
0                      116550
Entre 11 y 20 horas    115857
Entre 21 y 30 horas     92693
Menos de 10 horas       87191
Name: count, dtype: int64


In [ ]:
encoded_df = pd.get_dummies(df['ESTU_HORASSEMANATRABAJA'], prefix='Horas_Trabajo')

df = pd.concat([df, encoded_df], axis=1)

df.drop('ESTU_HORASSEMANATRABAJA', axis=1, inplace=True)


### Implementar codificación ordinal para el rendimiento:

In [ ]:
print(df['RENDIMIENTO_GLOBAL'].value_counts())

RENDIMIENTO_GLOBAL
alto          175619
bajo          172987
medio-bajo    172275
medio-alto    171619
Name: count, dtype: int64


In [ ]:
mapeo_rangos = {'bajo': 1, 'medio-bajo': 2, 'medio-alto': 3, 'alto': 4}
df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(mapeo_rangos)

In [ ]:
print(df['RENDIMIENTO_GLOBAL'].value_counts())

RENDIMIENTO_GLOBAL
4    175619
1    172987
2    172275
3    171619
Name: count, dtype: int64


## 5. Borrar datos repetidos:

In [ ]:
print(df['ESTU_PRGM_ACADEMICO'].value_counts())

ESTU_PRGM_ACADEMICO
DERECHO                                                   53244
ADMINISTRACION DE EMPRESAS                                51902
CONTADURIA PUBLICA                                        39664
PSICOLOGIA                                                31932
INGENIERIA INDUSTRIAL                                     28481
                                                          ...  
LICENCIATURA EN ESPAÑOL Y FILOLOGÍA                           1
ADMINISTRACION EN NEGOCIOS INTERNACIONALES                    1
ADMINISTRACION DE COMERCIO EXTERIOR                           1
LICENCIATURA EN EDUCACI¿N F¿SICA RECREACI¿N Y DEPORTES        1
PROFESIONAL EN INVESTIGACIÓN CRIMINAL                         1
Name: count, Length: 948, dtype: int64


In [ ]:
print(df['ESTU_PRGM_ACADEMICO'].unique())

In [ ]:
# Convertir todo el texto a minúsculas
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.lower()

# Aplicar limpieza de texto adicional si es necesario (eliminar caracteres especiales, etc.)
# Por ejemplo, eliminar acentos
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Mostrar algunos ejemplos de datos normalizados
print(df['ESTU_PRGM_ACADEMICO'].head())

In [ ]:
print(df['ESTU_PRGM_ACADEMICO'].unique())

## 6. Imputar Datos Faltantes:

### Imputar datos de columnas que tienen True/False:

In [ ]:
print(df.isnull().sum())

ID                                              0
PERIODO                                         0
ESTU_PRGM_ACADEMICO                             0
ESTU_PRGM_DEPARTAMENTO                          0
FAMI_ESTRATOVIVIENDA                        32137
FAMI_TIENEINTERNET                          26629
FAMI_EDUCACIONPADRE                         23178
FAMI_TIENELAVADORA                          39773
FAMI_TIENEAUTOMOVIL                         43623
ESTU_PAGOMATRICULAPROPIO                     6498
FAMI_TIENECOMPUTADOR                        38103
FAMI_EDUCACIONMADRE                         23664
RENDIMIENTO_GLOBAL                              0
Entre 1 millón y menos de 2.5 millones          0
Entre 2.5 millones y menos de 4 millones        0
Entre 4 millones y menos de 5.5 millones        0
Entre 5.5 millones y menos de 7 millones        0
Entre 500 mil y menos de 1 millón               0
Menos de 500 mil                                0
Más de 7 millones                               0


In [ ]:
columnas_booleanas = ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
                      'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']

for columna in columnas_booleanas:
    moda = df[columna].mode()[0]
    df[columna].fillna(moda, inplace=True)

In [ ]:
moda_estrato = df['FAMI_ESTRATOVIVIENDA'].mode()[0]  # Obtener la moda
df['FAMI_ESTRATOVIVIENDA'].fillna(moda_estrato, inplace=True)

### Padres y Madres

In [ ]:
madres = df['FAMI_EDUCACIONMADRE'].value_counts()
padres = df['FAMI_EDUCACIONPADRE'].value_counts()

print(madres)
print("---------------")
print(padres)

FAMI_EDUCACIONMADRE
Secundaria (Bachillerato) completa      141744
Primaria incompleta                      99420
Técnica o tecnológica completa           89542
Educación profesional completa           85326
Secundaria (Bachillerato) incompleta     81012
Primaria completa                        56125
Postgrado                                46246
Técnica o tecnológica incompleta         27533
Educación profesional incompleta         22470
Ninguno                                  14483
No sabe                                   3017
No Aplica                                 1918
Name: count, dtype: int64
---------------
FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      128289
Primaria incompleta                     125675
Educación profesional completa           83117
Secundaria (Bachillerato) incompleta     71654
Técnica o tecnológica completa           62995
Primaria completa                        55958
Postgrado                                44169
Educación profesional inc

In [ ]:
moda_padre = df['FAMI_EDUCACIONPADRE'].mode()[0]
df['FAMI_EDUCACIONPADRE'].fillna(moda_padre, inplace=True)

moda_madre = df['FAMI_EDUCACIONMADRE'].mode()[0]
df['FAMI_EDUCACIONMADRE'].fillna(moda_madre, inplace=True)

### Datos Restantes:


In [ ]:
moda_rendimiento = df['RENDIMIENTO_GLOBAL'].mode()[0]

df['RENDIMIENTO_GLOBAL'].fillna(moda_rendimiento, inplace=True)

In [ ]:
moda_programa = df['ESTU_PRGM_DEPARTAMENTO'].mode()[0]

df['ESTU_PRGM_DEPARTAMENTO'].fillna(moda_programa, inplace=True)


# Resultados (Por Ahora):

In [ ]:
# DATOS FALTANTES
print(df.isnull().sum())

ID                                          0
PERIODO                                     0
ESTU_PRGM_ACADEMICO                         0
ESTU_PRGM_DEPARTAMENTO                      0
FAMI_ESTRATOVIVIENDA                        0
FAMI_TIENEINTERNET                          0
FAMI_EDUCACIONPADRE                         0
FAMI_TIENELAVADORA                          0
FAMI_TIENEAUTOMOVIL                         0
ESTU_PAGOMATRICULAPROPIO                    0
FAMI_TIENECOMPUTADOR                        0
FAMI_EDUCACIONMADRE                         0
RENDIMIENTO_GLOBAL                          0
Entre 1 millón y menos de 2.5 millones      0
Entre 2.5 millones y menos de 4 millones    0
Entre 4 millones y menos de 5.5 millones    0
Entre 5.5 millones y menos de 7 millones    0
Entre 500 mil y menos de 1 millón           0
Menos de 500 mil                            0
Más de 7 millones                           0
No pagó matrícula                           0
Horas_Trabajo_0                   

In [ ]:
print(df.columns)

num_columnas = len(df.columns)
print("Número de columnas:", num_columnas)


Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
       'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones',
       'Entre 5.5 millones y menos de 7 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'Más de 7 millones', 'No pagó matrícula', 'Horas_Trabajo_0',
       'Horas_Trabajo_Entre 11 y 20 horas',
       'Horas_Trabajo_Entre 21 y 30 horas', 'Horas_Trabajo_Menos de 10 horas',
       'Horas_Trabajo_Más de 30 horas'],
      dtype='object')
Número de columnas: 26
